In [1]:
    sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1573547559816_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
spark.sql("""
select *, substr(response,1,1)
from 
`lovina_s3_etl`.`t_task`
where status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{')
limit 10
""").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+-------+-------+----------+--------------------+------+-----------+--------------------+--------------------+--------------------+---------+---------+-------------------------+
|     id|customer_id|loan_id|channel|channel_id|       thirdparty_id|status|retry_times|            response|         create_time|         update_time|client_id|query_key|substring(response, 1, 1)|
+-------+-----------+-------+-------+----------+--------------------+------+-----------+--------------------+--------------------+--------------------+---------+---------+-------------------------+
|3188745|    3222541|   null|     20|   3222541|TASKSOCIAL1030012...|    40|       null|{"code":0,"data":...|2019-09-17 11:06:...|2019-09-17 11:07:...|        0|     null|                        {|
|3188801|    2112889|   null|     20|   2112889|TASKSOCIAL1030012...|    40|       null|{"code":0,"data":...|2019-09-17 11:11:...|2019-09-17 11:12:...|        0|     null|                        {|
|3184646| 

In [3]:
df1=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.code") as code,
get_json_object(response,"$.message") as message,
get_json_object(response,"$.task_id") as task_id,
get_json_object(response,"$.data.identity_code") as identity_code,
get_json_object(response,"$.data.channel_src") as channel_src,
get_json_object(response,"$.data.user_mobile") as user_mobile,
get_json_object(response,"$.data.real_name") as real_name,
get_json_object(response,"$.data.channel_code") as channel_code,
get_json_object(response,"$.data.channel_type") as channel_type,
get_json_object(response,"$.data.lost_data") as lost_data,
get_json_object(response,"$.data.task_data.base_info.hometown") as hometown,
get_json_object(response,"$.data.task_data.base_info.birthdate") as birthdate,
get_json_object(response,"$.data.task_data.base_info.birthyear") as birthyear,
get_json_object(response,"$.data.task_data.base_info.gender") as gender,
get_json_object(response,"$.data.task_data.base_info.friendsnum") as friendsnum,
get_json_object(response,"$.data.task_data.base_info.language") as language,
replace(substr(get_json_object(response,"$.data.task_data.base_info.emails"),2,length(get_json_object(response,"$.data.task_data.base_info.emails"))-2),"\\"","") as emails,
get_json_object(response,"$.data.task_data.base_info.political_trend") as political_trend,
get_json_object(response,"$.data.task_data.base_info.relationshipstatus") as relationshipstatus,
get_json_object(response,"$.data.task_data.base_info.blood_type") as blood_type,
get_json_object(response,"$.data.task_data.base_info.currentcity") as currentcity,
get_json_object(response,"$.data.task_data.base_info.nickname") as nickname,
get_json_object(response,"$.data.task_data.base_info.religion_trend") as religion_trend,
replace(substr(get_json_object(response,"$.data.task_data.base_info.mobilephones"),2,length(get_json_object(response,"$.data.task_data.base_info.mobilephones"))-2),"\\"","") as mobilephones,
get_json_object(response,"$.data.task_data.base_info.payments_info") as payments_info,
get_json_object(response,"$.data.task_data.base_info.userid") as userid,
get_json_object(response,"$.data.channel_attr") as channel_attr,
get_json_object(response,"$.data.created_time") as created_time,
get_json_object(response,"$.data.user_name") as user_name,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' ) 
and customer_id is not null
and id<11773266
""")
df1.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df1=spark.sql("""
select 
id,
year(to_date(create_time)) as year,
month(to_date(create_time)) as month,
day(to_date(create_time)) as day
from `lovina_s3_etl`.`t_task`
where status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' ) 
and customer_id is not null
and  to_date(create_time)  ='2019-12-12' 
and id<9247446
""").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+-----+---+
| id|year|month|day|
+---+----+-----+---+
+---+----+-----+---+

In [5]:
df2=spark.sql("""
select 
get_json_object(concat(a.friends_info,'}'),'$.nickname') as nickname,
get_json_object(concat(a.friends_info,'}'),'$.mutual_friends') as mutual_friends,
thirdparty_id,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,a.query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,friends_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.friends_info"),2,length(get_json_object(response,"$.data.task_data.friends_info"))-2),'},')) as  friends_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )
and customer_id is not null
and id<11773266
) a 
""")
df2.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_friends_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

df3=spark.sql("""
select 
get_json_object(concat(a.family_info,'}'),'$.name') as name,
get_json_object(concat(a.family_info,'}'),'$.relationship') as relationship,
a.thirdparty_id,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,family_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.family_info"),2,length(get_json_object(response,"$.data.task_data.family_info"))-2),'},')) as  family_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )
and customer_id is not null
and id<11773266
) a 
""")
df3.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_family_info/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df4=spark.sql("""
select 
get_json_object(concat(a.college_info,'}'),'$.college_graduation') as college_graduation,
get_json_object(concat(a.college_info,'}'),'$.college_name') as college_name,
a.thirdparty_id,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,college_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.education_info.college_info"),2,length(get_json_object(response,"$.data.task_data.education_info.college_info"))-2),'},')) as college_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' ) 
and customer_id is not null
and id<11773266
) a 
""")
df4.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_education_college_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df5=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.highschool_info,'}'),'$.highschool_graduation') as highschool_graduation,
get_json_object(concat(a.highschool_info,'}'),'$.highschool_name') as highschool_name,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,highschool_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.education_info.highschool_info"),2,length(get_json_object(response,"$.data.task_data.education_info.highschool_info"))-2),'},')) as highschool_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )   and id<11773266
and customer_id is not null
) a 
""")
df5.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_education_highschool_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df6=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.messenger_info,'}'),'$.friend_nickname') as friend_nickname,
get_json_object(concat(a.messenger_info,'}'),'$.chattime') as chattime,
get_json_object(concat(a.messenger_info,'}'),'$.content') as content,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,messenger_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.messenger_info"),2,length(get_json_object(response,"$.data.task_data.messenger_info"))-2),'},')) as messenger_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )   and id<11773266
and customer_id is not null
) a 
""")
df6.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_messenger_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df7=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.work_info,'}'),'$.finishtime') as finishtime,
get_json_object(concat(a.work_info,'}'),'$.city') as city,
get_json_object(concat(a.work_info,'}'),'$.company') as company,
get_json_object(concat(a.work_info,'}'),'$.position') as position,
get_json_object(concat(a.work_info,'}'),'$.starttime') as starttime,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,work_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.work_info"),2,length(get_json_object(response,"$.data.task_data.work_info"))-2),'},')) as work_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )   and id<11773266
and customer_id is not null
) a 
""")
df7.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_work_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df8=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.friends_publish_info,'}'),'$.site') as site,
get_json_object(concat(a.friends_publish_info,'}'),'$.page_time') as page_time,
get_json_object(concat(a.friends_publish_info,'}'),'$.nickname') as nickname,
get_json_object(concat(a.friends_publish_info,'}'),'$.commentsnum') as commentsnum,
get_json_object(concat(a.friends_publish_info,'}'),'$.likenum') as likenum,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,friends_publish_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.friends_publish_info"),2,length(get_json_object(response,"$.data.task_data.friends_publish_info"))-2),'},')) as friends_publish_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )  and id<11773266
and customer_id is not null
) a 
""")
df8.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_friends_publish_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df9=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.historical_login_info,'}'),'$.page_time') as page_time,
get_json_object(concat(a.historical_login_info,'}'),'$.locus') as locus,
get_json_object(concat(a.historical_login_info,'}'),'$.tool') as tool,
get_json_object(concat(a.historical_login_info,'}'),'$.way') as way,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,historical_login_info
from `lovina_s3_etl`.`t_task_1` a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.historical_login_info"),2,length(get_json_object(response,"$.data.task_data.historical_login_info"))-2),'},')) as historical_login_info
where  status=40 and channel=20  and (response is null or response ='' or substr(response,1,1)='{' )  and id<11773266 
and customer_id is not null
) a 
""")
df9.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_historical_login_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
df10=spark.sql("""
select 
a.thirdparty_id,
get_json_object(concat(a.publish_info,'}'),'$.site') as site,
get_json_object(concat(a.publish_info,'}'),'$.page_time') as page_time,
get_json_object(concat(a.publish_info,'}'),'$.nickname') as nickname,
substr(get_json_object(concat(a.publish_info,'}'),'$.likenames'),2,length(get_json_object(concat(a.publish_info,'}'),'$.likenames'))-2)as likenames,
get_json_object(concat(a.publish_info,'}'),'$.commentsnum') as commentsnum,
get_json_object(concat(a.publish_info,'}'),'$.likenum') as likenum,
cast(a.customer_id as string )as customer_id,
cast(a.loan_id as string )as loan_id,
cast(a.create_time as string) as create_time,query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from (
select  *,publish_info
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.task_data.publish_info"),2,length(get_json_object(response,"$.data.task_data.publish_info"))-2),'},')) as publish_info
where  status=40 and channel=20 and customer_id is not null and (response is null or response ='' or substr(response,1,1)='{' )  and id<11773266 
) a 
""")
df10.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_facebook_publish_info/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
thirdparty_tongdun_guard=spark.sql("""
select id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string ) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,'$.success') as success,
cast(get_json_object(response,'$.result_desc.ANTIFRAUD.final_score') as int ) as final_score,
get_json_object(response,'$.result_desc.ANTIFRAUD.final_decision') as final_decision,
get_json_object(response,'$.result_desc.INFOANALYSIS.address_detect.true_ip_address') as address_detect_true_ip_address,
get_json_object(response,'$.result_desc.INFOANALYSIS.device_info.error') as device_info_error,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1` a
where  status=40 
and channel=40  
and customer_id is not null and (response is null or response ='' or substr(response,1,1)='{' ) 
and  id<11773266 
and (client_id is null or client_id =0)
""")
thirdparty_tongdun_guard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from urllib.request import urlopen
import json
from pprint import pprint
from pyspark.sql import types as T

# a='{"success":true,"id":"WF2019021715404113217023","result_desc":{"ANTIFRAUD":{"final_score":0,"risk_items":[{"rule_id":"23374771","score":0,"decision":"Accept","risk_name":"Borrower\\'s,name_same_with_KTP_name","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"},{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"}],"type":"mh_detail"}]},{"rule_id":"23374772","score":0,"decision":"Accept1111","risk_name":"Borrower_name_same_with_KTP_name111111","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name111111","hit_rule_type":"Identity verification11111","field_type":"name11111","input_value":"Mertina Zai1111"},{"description":"Borrowers name is same with KTP name1111","hit_rule_type":"Identity verification11111","field_type":"name1111","input_value":"Mertina Zai11111"}],"type":"mh_detail"}]}],"final_decision":"Accept"}}}'
# def tongdun_guard_json(b,typeValue):
#     return getJsonArray(b,typeValue);
def tongdun_guard_json(b,typeValue):
    risk_itemls=[];
    risk_detaillist=[];
    risk_detailslist=[];
    text = json.loads(b)
    for field in text.keys():
        if field=="risk_items":
            index=0;
            for v1 in text[field]:
                index=index+1;
                for field1 in list(v1):
                    if field1=='risk_detail':
                        for v2 in v1[field1]:
                            for risk_detailfield in  list(v2):
                                if risk_detailfield=='risk_details':
                                     for v3 in v2[risk_detailfield]:
                                         v3["rule_id"]=v1["rule_id"];
                                         v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                         risk_detailslist.append(json.dumps(v3));
                                     v2.pop(risk_detailfield);
                                v2["rule_id"]=v1["rule_id"];
                                v2["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                risk_detaillist.append(json.dumps(v2))
                        v1.pop(field1);
                risk_itemls.append(json.dumps(v1))
            break;
        elif field!="risk_items" and type(text[field]) == dict:
            result_desc=json.dumps(text[field]);
            return tongdun_guard_json(result_desc,typeValue);
        else :
            continue
    if typeValue=='risk_items':
        return  risk_itemls
    elif typeValue=='risk_detail':
        return  risk_detaillist
    elif typeValue=='risk_details':
        return risk_detailslist;
# tongdun_guard_json(a,"risk_items");
spark.udf.register("tongdun_guard_json",tongdun_guard_json,T.ArrayType(T.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function tongdun_guard_json at 0x7f7d685361e0>

In [19]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
thirdparty_tongdun_guard=spark.sql("""
select id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string ) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,'$.success') as success,
cast(get_json_object(response,'$.result_desc.ANTIFRAUD.final_score') as int ) as final_score,
get_json_object(response,'$.result_desc.ANTIFRAUD.final_decision') as final_decision,
get_json_object(response,'$.result_desc.INFOANALYSIS.address_detect.true_ip_address') as address_detect_true_ip_address,
get_json_object(response,'$.result_desc.INFOANALYSIS.device_info.error') as device_info_error,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from `lovina_s3_etl`.`t_task_1`a
where  status=40 
and channel=40  
and customer_id is not null and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and (client_id is null or client_id =0)
""")
thirdparty_tongdun_guard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
thirdparty_tongdun_guard=spark.sql("""
select count(1)
from `lovina_s3_etl`.`t_task`a 
where to_date(create_time)  ='2019-11-25' 
and  status=40 
and channel=40  
and customer_id is not null and (response is null or response ='' or substr(response,1,1)='{' ) 
and (client_id is null or client_id =0)
""").show()
# thirdparty_tongdun_guard.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|       1|
+--------+

In [3]:
thirdparty_tongdun_guard_risk_items=spark.sql("""
select 
get_json_object(risk_items,"$.rule_id") as rule_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
cast(get_json_object(risk_items,"$.score")as int) as score,
get_json_object(risk_items,"$.decision") as decision,
get_json_object(risk_items,"$.risk_name") as risk_name,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_items as risk_items
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_json(response,"risk_items")) as risk_items
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and create_time > cast('2020-01-01 00:00:00'  AS timestamp) 
)  a
""")
thirdparty_tongdun_guard_risk_items.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_risk_items/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
thirdparty_tongdun_guard_risk_items=spark.sql("""
select 
get_json_object(risk_items,"$.rule_id") as rule_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
cast(get_json_object(risk_items,"$.score")as int) as score,
get_json_object(risk_items,"$.decision") as decision,
get_json_object(risk_items,"$.risk_name") as risk_name,
year(to_date(create_time)) as year,
month(to_date(create_time)) as month,
day(to_date(create_time)) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_items as risk_items
from `lovina_s3_etl`.`t_task`
LATERAL VIEW explode(tongdun_guard_json(response,"risk_items")) as risk_items
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and  id<11773266 
limit 100
)  a
""").show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+-------+--------------------+---------+--------------------+-----+--------+--------------------+----+-----+---+
| rule_id|customer_id|loan_id|         create_time|query_key|       thirdparty_id|score|decision|           risk_name|year|month|day|
+--------+-----------+-------+--------------------+---------+--------------------+-----+--------+--------------------+----+-----+---+
|23374971|    2955133|5617575|2019-02-27 01:46:...|     null|WF201902270846101...|    5|  Accept|Borrower_applied_...|2019|    2| 27|
|23375001|    2955133|5617575|2019-02-27 01:46:...|     null|WF201902270846101...|    5|  Accept|Borrower_applied_...|2019|    2| 27|
|23375031|    2955133|5617575|2019-02-27 01:46:...|     null|WF201902270846101...|    5|  Accept|Borrower_applied_...|2019|    2| 27|
|23374771|    2955133|5617575|2019-02-27 01:46:...|     null|WF201902270846101...|    0|  Accept|Borrower_name_sam...|2019|    2| 27|
|23375031|    2027836|5903205|2019-03-30 03:57:...|     null|W

In [26]:
thirdparty_tongdun_guard_risk_detail=spark.sql("""
select 
get_json_object(risk_detail,"$.rule_id") as rule_id,
get_json_object(risk_detail,"$.rule_detail_id") as rule_detail_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
get_json_object(risk_detail,"$.type") as type,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_detail as risk_detail
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_json(response,"risk_detail")) as risk_detail
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_risk_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_risk_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
thirdparty_tongdun_guard_risk_details=spark.sql("""
select 
get_json_object(risk_details,"$.rule_id") as rule_id,
get_json_object(risk_details,"$.rule_detail_id") as rule_detail_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
get_json_object(risk_details,"$.description") as description,
get_json_object(risk_details,"$.hit_rule_type") as hit_rule_type,
get_json_object(risk_details,"$.field_type") as field_type,
get_json_object(risk_details,"$.input_value") as input_value,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_details as risk_details
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_json(response ,"risk_details")) as risk_details
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_risk_details.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_risk_details/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
from urllib.request import urlopen
import json
from pprint import pprint
from pyspark.sql import types as T
# a='{"success":true,"id":"WF2019101615354013301181","result_desc":{"INFOANALYSIS":{"address_detect":{"true_ip_address":null},"device_info":{"error":"Parameters are not complete."}},"ANTIFRAUD":{"final_score":20,"risk_items":[{"rule_id":"23408122","score":9,"decision":"Accept","risk_name":"3个月内身份证关联多个手机号或邮箱_90d_id_number_relates_to_email_or_mobile","risk_detail":[{"frequency_detail_list":[{"data":["09507861250","956※※※※190","09561194190"],"detail":"90d_id_mobile_loan_all:3"}],"type":"frequency_detail"}]},{"rule_id":"23408132","score":6,"decision":"Accept","risk_name":"3个月内手机号或邮箱关联多个身份证_90d_mobile_email_relates_to_more_than_one_id_number","risk_detail":[{"frequency_detail_list":[{"data":["749758663000","020※※※※418"],"detail":"90d_mobile_id_loan_all:2"}],"type":"frequency_detail"}]},{"rule_id":"23408352","score":3,"decision":"Accept","risk_name":"30天内申请人在多个平台申请借款_Borrower_applied_loan_in_many_platforms_in_30_days","risk_detail":[{"platform_detail_dimension":[{"count":1,"detail":[{"count":1,"industry_display_name":"无行业"}],"dimension":"account_mobile"},{"count":2,"detail":[{"count":2,"industry_display_name":"无行业"}],"dimension":"id_number"}],"platform_detail":[{"count":2,"industry_display_name":"无行业"}],"description":"30d_ids_multiplatforms_loan_all","type":"platform_detail","platform_count":2}]},{"rule_id":"23408372","score":2,"decision":"Accept","risk_name":"90天内申请人在多个平台申请借款_Borrower_applied_loan_in_many_platforms_in_90_days","risk_detail":[{"platform_detail_dimension":[{"count":2,"detail":[{"count":2,"industry_display_name":"无行业"}],"dimension":"account_mobile"},{"count":2,"detail":[{"count":2,"industry_display_name":"无行业"}],"dimension":"id_number"}],"platform_detail":[{"count":3,"industry_display_name":"无行业"}],"description":"90d_ids_multiplatforms_loan_all","type":"platform_detail","platform_count":3}]}],"final_decision":"Review"}}}'
# def tongdun_guard_json(b,typeValue):
#     return getJsonArray(b,typeValue);
def tongdun_guard_Republika_json(b,typeValue):
    risk_itemls=[];
    risk_detaillist=[];
    risk_detailslist=[];
    frequency_detail_list=[];
    platform_detail_dimension=[];
    platform_detail_dimension_detail=[];
    platform_detail=[];
    text = json.loads(b)
    for field in text.keys():
        if field=="risk_items":
            index=0;
            for v1 in text[field]:
                index=index+1;
                for field1 in list(v1):
                    if field1=='risk_detail':
                        for v2 in v1[field1]:
                            for risk_detailfield in  list(v2):
                                if risk_detailfield=='risk_details':
                                    for v3 in v2[risk_detailfield]:
                                        v3["rule_id"]=v1["rule_id"];
                                        v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                        risk_detailslist.append(json.dumps(v3));
                                    v2.pop("risk_details");
                                if risk_detailfield=='frequency_detail_list':
                                    for v3 in v2[risk_detailfield]:
                                        v3["rule_id"]=v1["rule_id"];
                                        v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                        frequency_detail_list.append(json.dumps(v3));
                                    v2.pop("frequency_detail_list");
                                if risk_detailfield=='platform_detail_dimension':
                                    index2=0
                                    for v3 in v2[risk_detailfield]:
                                        index2=index2+1;
                                        v3["rule_id"]=v1["rule_id"];
                                        v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                        v3["rule_detail_id2"]=v1["rule_id"]+"_"+str(index)+"_"+str(index2);
                                        if v3["detail"]:
                                            for v4 in v3["detail"]:
                                                v4["rule_id"]=v1["rule_id"];
                                                v4["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                                v4["rule_detail_id2"]=v1["rule_id"]+"_"+str(index)+"_"+str(index2);
                                                v3.pop("detail");
                                            platform_detail_dimension_detail.append(json.dumps(v4));
                                        platform_detail_dimension.append(json.dumps(v3));
                                    v2.pop("platform_detail_dimension");
                                if risk_detailfield=='platform_detail':
                                    for v3 in v2[risk_detailfield]:
                                        index2=index2+1;
                                        v3["rule_id"]=v1["rule_id"];
                                        v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                    # print(v3)
                                    v2.pop("platform_detail");
                                    platform_detail.append(json.dumps(v3));
                            v2["rule_id"]=v1["rule_id"];
                            v2["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                            risk_detaillist.append(json.dumps(v2))
                        v1.pop(field1);
                risk_itemls.append(json.dumps(v1))
            break;
        elif field!="risk_items" and field!="INFOANALYSIS" and type(text[field]) == dict:
            result_desc=json.dumps(text[field]);
            return tongdun_guard_Republika_json(result_desc,typeValue);
        else :
             continue
    if typeValue=='risk_items':
        return  risk_itemls
    elif typeValue=='risk_detail':
        return  risk_detaillist
    elif typeValue=='risk_details':
        return risk_detailslist;
    elif typeValue=='frequency_detail_list':
        return  frequency_detail_list
    elif typeValue=='platform_detail_dimension':
        return  platform_detail_dimension
    elif typeValue=='platform_detail_dimension_detail':
        return platform_detail_dimension_detail;
    elif typeValue=='platform_detail':
        return platform_detail;
# tongdun_guard_Republika_json(a,"risk_items");
spark.udf.register("tongdun_guard_Republika_json",tongdun_guard_Republika_json,T.ArrayType(T.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function tongdun_guard_Republika_json at 0x7fc6e1ec8b70>

In [32]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
thirdparty_tongdun_guard_Republika=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string ) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,'$.success') as success,
cast(get_json_object(response,'$.result_desc.ANTIFRAUD.final_score') as int ) as final_score,
get_json_object(response,'$.result_desc.ANTIFRAUD.final_decision') as final_decision,
get_json_object(response,'$.result_desc.INFOANALYSIS.address_detect.true_ip_address') as address_detect_true_ip_address,
get_json_object(response,'$.result_desc.INFOANALYSIS.device_info.error') as device_info_error,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`a
where  status=40 
and channel=40  
and customer_id is not null and (response is null or response ='' or substr(response,1,1)='{' ) 
and client_id=2  
and  id<11773266 
""")
thirdparty_tongdun_guard_Republika.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_Republika/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
thirdparty_tongdun_guard_risk_items_republika=spark.sql("""
select 
get_json_object(risk_items,"$.rule_id") as rule_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
get_json_object(risk_items,"$.score") as score,
get_json_object(risk_items,"$.decision") as decision,
get_json_object(risk_items,"$.risk_name") as risk_name,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_items as risk_items
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"risk_items")) as risk_items
where status=40 
and channel=40  
and customer_id is not null
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_risk_items_republika.write.mode("overwrite").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_risk_items_Republika/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
thirdparty_tongdun_guard_risk_detail_republika=spark.sql("""
select 
get_json_object(risk_detail,"$.rule_id") as rule_id,
get_json_object(risk_detail,"$.rule_detail_id") as rule_detail_id,
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
get_json_object(risk_detail,"$.type") as type,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
risk_detail as risk_detail
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"risk_detail")) as risk_detail
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and customer_id is not null
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_risk_detail_republika.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_risk_detail_Republika/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|     705|
+--------+

In [35]:
thirdparty_tongdun_guard_frequency_detail=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.query_key,
a.thirdparty_id,
get_json_object(frequency_detail_list,"$.rule_id") as rule_id,
get_json_object(frequency_detail_list,"$.rule_detail_id") as rule_detail_id,
substr(get_json_object(frequency_detail_list,"$.data"),2,length(get_json_object(frequency_detail_list,"$.data"))-2) as data,
get_json_object(frequency_detail_list,"$.detail") as detail,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
frequency_detail_list as frequency_detail_list
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"frequency_detail_list")) as frequency_detail_list
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and  id<11773266 
and customer_id is not null
)  a 
""")
thirdparty_tongdun_guard_frequency_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_frequency_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
list1=spark.sql("""
select 
count(1)
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
frequency_detail_list as frequency_detail_list
from `lovina_s3_etl`.`t_task`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"frequency_detail_list")) as frequency_detail_list
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and customer_id is not null
)  a 
""").show()
# thirdparty_tongdun_guard_frequency_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_frequency_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|   97517|
+--------+

In [36]:
thirdparty_tongdun_guard_platform_detail=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.thirdparty_id,
a.query_key,
get_json_object(frequency_detail_list,"$.rule_id") as rule_id,
get_json_object(frequency_detail_list,"$.rule_detail_id") as rule_detail_id,
cast(get_json_object(frequency_detail_list,"$.count") as int) as count,
get_json_object(frequency_detail_list,"$.industry_display_name") as industry_display_name,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
frequency_detail_list as frequency_detail_list
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"platform_detail")) as frequency_detail_list
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and customer_id is not null
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_platform_detail.write.mode("overwrite").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_platform_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
thirdparty_tongdun_guard_platform_detail_dimension=spark.sql("""
select
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.thirdparty_id,
a.query_key,
get_json_object(platform_detail_dimension,"$.rule_id") as rule_id,
get_json_object(platform_detail_dimension,"$.rule_detail_id") as rule_detail_id,
cast(get_json_object(platform_detail_dimension,"$.count") as int) as count,
get_json_object(platform_detail_dimension,"$.dimension") as dimension,
get_json_object(platform_detail_dimension,"$.rule_detail_id2") as rule_detail_id2,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
platform_detail_dimension as platform_detail_dimension
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"platform_detail_dimension")) as platform_detail_dimension
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and customer_id is not null
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_platform_detail_dimension.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_platform_detail_dimension/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
thirdparty_tongdun_guard_platform_detail_dimension_detail=spark.sql("""
select
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.thirdparty_id,
a.query_key,
get_json_object(platform_detail_dimension_detail,"$.rule_id") as rule_id,
get_json_object(platform_detail_dimension_detail,"$.rule_detail_id") as rule_detail_id,
cast(get_json_object(platform_detail_dimension_detail,"$.count") as int) as count,
get_json_object(platform_detail_dimension_detail,"$.industry_display_name") as industry_display_name,
get_json_object(platform_detail_dimension_detail,"$.rule_detail_id2") as rule_detail_id2,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key,
thirdparty_id,
platform_detail_dimension_detail as platform_detail_dimension_detail
from `lovina_s3_etl`.`t_task_1`
LATERAL VIEW explode(tongdun_guard_Republika_json(response,"platform_detail_dimension_detail")) as platform_detail_dimension_detail
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and client_id=2 
and customer_id is not null
and  id<11773266 
)  a
""")
thirdparty_tongdun_guard_platform_detail_dimension_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_guard_platform_detail_dimension_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
1111111111111111111111111111111111111111111111111111111111111111111111111111111

In [39]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
thirdparty_tongdun_ktp_ocr=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.code") as code,
get_json_object(response,"$.message") as message,
get_json_object(response,"$.task_id") as task_id,
get_json_object(response,"$.data.identity_code") as identity_code,
get_json_object(response,"$.data.created_time") as created_time,
get_json_object(response,"$.data.channel_src") as channel_src,
get_json_object(response,"$.data.user_mobile") as user_mobile,
get_json_object(response,"$.data.user_name") as user_name,
get_json_object(response,"$.data.real_name") as real_name,
get_json_object(response,"$.data.channel_code") as channel_code,
get_json_object(response,"$.data.channel_type") as channel_type,
get_json_object(response,"$.data.channel_attr") as channel_attr,
get_json_object(response,"$.data.lost_data") as lost_data,
replace(substr(get_json_object(response,"$.data.task_data.other.img_identity_url"),2,length(get_json_object(response,"$.data.task_data.other.img_identity_url"))-2),"\\"","") as img_identity_url,
get_json_object(response,"$.data.task_data.profile.birthdate") as birthday,
get_json_object(response,"$.data.task_data.profile.address") as address,
get_json_object(response,"$.data.task_data.profile.occupation") as occupation,
get_json_object(response,"$.data.task_data.profile.gender") as gender,
get_json_object(response,"$.data.task_data.profile.expiry_date") as expiry_date,
get_json_object(response,"$.data.task_data.profile.religion") as religion,
get_json_object(response,"$.data.task_data.profile.nik") as nik,
get_json_object(response,"$.data.task_data.profile.marital_status") as marital_status,
get_json_object(response,"$.data.task_data.profile.birthplace") as birthplace,
get_json_object(response,"$.data.task_data.profile.nationality") as nationality,
get_json_object(response,"$.data.task_data.profile.street") as street,
get_json_object(response,"$.data.task_data.profile.blood_type") as blood_type,
get_json_object(response,"$.data.task_data.profile.name") as name,
get_json_object(response,"$.data.task_data.profile.village") as village,
get_json_object(response,"$.data.task_data.profile.rt_rw") as rt_rw,
get_json_object(response,"$.data.task_data.return_info.ktp_result.wrong_type") as wrong_type,
get_json_object(response,"$.data.task_data.return_info.is_match") as is_match,
get_json_object(response,"$.data.task_data.return_info.phone_is_match") as phone_is_match,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=50  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_tongdun_ktp_ocr.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_tongdun_ktp_ocr/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
thirdparty_advance_ai_multi_platform=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.code") as code,
get_json_object(response,"$.message") as message,
get_json_object(response,"$.extra") as extra,
get_json_object(response,"$.transactionId") as transactionid,
get_json_object(response,"$.pricingStrategy") as pricingstrategy,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=60  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_advance_ai_multi_platform.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_advance_ai_multi_platform/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
thirdparty_advance_ai_multi_platform_statisticcustomerinfo=spark.sql("""
select 
cast(a.customer_id as string) as customer_id, cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,a.thirdparty_id,query_key,
get_json_object(concat(a.statistics,'}'),'$.timePeriod') as timeperiod,
cast(get_json_object(concat(a.statistics,'}'),'$.queryCount') as int )as querycount,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select customer_id,loan_id,create_time,thirdparty_id,query_key,statistics
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.statistics.statisticCustomerInfo"),2,length(get_json_object(response,"$.data.statistics.statisticCustomerInfo"))-2),'},')) as statistics
where  status=40 and channel=60  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
) a 
""")
thirdparty_advance_ai_multi_platform_statisticcustomerinfo.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_advance_ai_multi_platform_statisticCustomerInfo/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
thirdparty_advance_ai_multi_platform_lastTwoWeeksQueryInfo=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,a.thirdparty_id,query_key,
cast(get_json_object(concat(a.lastTwoWeeksQueryInfo,'}'),'$.times') as int) as times,
get_json_object(concat(a.lastTwoWeeksQueryInfo,'}'),'$.timeSlice') as timeslice,
cast(get_json_object(concat(a.lastTwoWeeksQueryInfo,'}'),'$.maxQueryCount') as int) as maxquerycount,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select customer_id,loan_id,create_time,thirdparty_id,query_key,lastTwoWeeksQueryInfo
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.statistics.lastTwoWeeksQueryInfo"),2,length(get_json_object(response,"$.data.statistics.lastTwoWeeksQueryInfo"))-2),'},')) as lastTwoWeeksQueryInfo
where  status=40 and channel=60  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
) a 
""")
thirdparty_advance_ai_multi_platform_lastTwoWeeksQueryInfo.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_advance_ai_multi_platform_lastTwoWeeksQueryInfo/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
thirdparty_advance_ai_multi_platform_records=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,a.thirdparty_id,query_key,
replace(substr(get_json_object(concat(a.records,'}'),'$.queryDates'),2,length(get_json_object(concat(a.records,'}'),'$.queryDates'))-2),"\\"","") as querydates,
cast(get_json_object(concat(a.records,'}'),'$.queryCount')as int) as querycount,
get_json_object(concat(a.records,'}'),'$.type') as type,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select customer_id,loan_id,create_time,thirdparty_id,query_key,records
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.data.records"),2,length(get_json_object(response,"$.data.records"))-2),'},')) as records
where  status=40 and channel=60  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
) a 
""")
thirdparty_advance_ai_multi_platform_records.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_advance_ai_multi_platform_records/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
thirdparty_izi_phone_verify=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
get_json_object(response,"$.message") as message,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=70  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_izi_phone_verify.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_izi_phone_verify/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
thirdparty_izi_phone_age=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
get_json_object(response,"$.message.phone") as phone,
get_json_object(response,"$.message.age") as age,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as message,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=80  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_izi_phone_age.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_izi_phone_age/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
thirdparty_izi_phone_whatsapp=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
get_json_object(response,"$.message.number") as number,
get_json_object(response,"$.mes as timesage.whatsapp") as whatsapp,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as message,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=90  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266  
and  customer_id is not null 
""")
thirdparty_izi_phone_whatsapp.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_izi_phone_whatsapp/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
thirdparty_ktp_multi_platfrom_detection=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
cast(get_json_object(response,"$.status")as int) as res_status,
cast(get_json_object(response,"$.message.total") as int) as total,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as message,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=200  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_ktp_multi_platfrom_detection.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_ktp_multi_platfrom_detection/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
thirdparty_ktp_multi_platform_detection_brief_check=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as message,
cast(get_json_object(response,"$.message.07d") as int) as `07d`,
cast(get_json_object(response,"$.message.14d") as int) as `14d`,
cast(get_json_object(response,"$.message.21d") as int) as `21d`,
cast(get_json_object(response,"$.message.30d") as int) as `30d`,
cast(get_json_object(response,"$.message.60d") as int) as `60d`,
cast(get_json_object(response,"$.message.90d") as int) as `90d`,
cast(get_json_object(response,"$.message.total") as int) as total,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=210  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_ktp_multi_platform_detection_brief_check.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_ktp_multi_platform_detection_brief_check/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
thirdparty_izi_black_check=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as messagem,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=220  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_izi_black_check.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_izi_black_check/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
thirdparty_identity_check=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.status") as res_status,
get_json_object(response,"$.message.msg") as msg,
get_json_object(response,"$.message.sourcedata") as sourcedata,
get_json_object(response,"$.message.name") as name,
get_json_object(response,"$.message.match") as match,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=230  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_identity_check.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_identity_check/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
thirdparty_yitu_face_verify=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
cast(client_id as String) as client_id,
query_key,
get_json_object(response,"$.rtn") as rtn,
get_json_object(response,"$.global_request_id") as global_request_id,
get_json_object(response,"$.message") as message,
cast(get_json_object(response,"$.pair_verify_result")  as int) as pair_verify_result,
cast(get_json_object(response,"$.pair_verify_similarity")  as double) as pair_verify_similarity,
get_json_object(response,"$.query_image_package_result.customer_defined_content") as customer_defined_content,
replace(substr(get_json_object(response,"$.query_image_package_result.query_image_contents"),2,length(get_json_object(response,"$.query_image_package_result.query_image_contents"))-2),"\\"","") as query_image_contents,
get_json_object(response,"$.query_image_package_result.is_same_person") as is_same_person,
get_json_object(response,"$.query_image_package_result.is_anti_screen_check_passed") as is_anti_screen_check_passed,
cast(get_json_object(response,"$.query_image_package_result.is_anti_screen_check_score") as double) as is_anti_screen_check_score,
get_json_object(response,"$.query_image_package_result.is_dark_illumination_check_passed") as is_dark_illumination_check_passed,
cast(get_json_object(response,"$.query_image_package_result.is_dark_illumination_check_score") as double) as is_dark_illumination_check_score,
get_json_object(response,"$.query_image_package_result.is_anti_picture_check_valid") as is_anti_picture_check_valid,
get_json_object(response,"$.query_image_package_result.is_anti_picture_check_passed") as is_anti_picture_check_passed,
cast(get_json_object(response,"$.query_image_package_result.is_anti_picture_check_score") as double) as is_anti_picture_check_score,
get_json_object(response,"$.query_image_package_result.is_anti_eye_blockage_check_valid") as is_anti_eye_blockage_check_valid,
get_json_object(response,"$.query_image_package_result.is_anti_eye_blockage_check_passed") as is_anti_eye_blockage_check_passed,
cast(get_json_object(response,"$.query_image_package_result.is_anti_eye_blockage_check_score") as double)as is_anti_eye_blockage_check_score,
get_json_object(response,"$.query_image_package_result.is_anti_hole_check_valid") as is_anti_hole_check_valid,
get_json_object(response,"$.query_image_package_result.is_anti_hole_check_passed") as is_anti_hole_check_passed,
cast(get_json_object(response,"$.query_image_package_result.is_anti_hole_check_score") as double) as is_anti_hole_check_score,
get_json_object(response,"$.verify_detail") as verify_detail,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=240  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_yitu_face_verify.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_yitu_face_verify/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
thirdparty_sti_mobile_check=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
get_json_object(response,"$.data.totalCount") as totalcount,
get_json_object(response,"$.result.code") as code,
get_json_object(response,"$.result.desc") as desc,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=250  and (response is null or response ='' or substr(response,1,1)='{' )  and  id<11773266 
and  customer_id is not null 
""")
thirdparty_sti_mobile_check.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_sti_mobile_check/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
thirdparty_sti_mobile_check_detail=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
a.thirdparty_id,
cast(a.create_time as string) as create_time,query_key,
get_json_object(concat(a.list,'}'),'$.msisdn') as msisdn,
get_json_object(concat(a.list,'}'),'$.status') as status,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select customer_id,loan_id,create_time,thirdparty_id,query_key,list
from `lovina_s3_etl`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.list"),2,length(get_json_object(response,"$.list"))-2),'},')) as list
where  status=40 and channel=250  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
) a 
""")
thirdparty_sti_mobile_check_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_sti_mobile_check_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
thirdparty_star_detector_apply_in_7_days=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
query_key,
cast(get_json_object(response,"$.id") as long) as res_id,
get_json_object(response,"$.reportno") as reportno,
get_json_object(response,"$.mobile") as mobile,
get_json_object(response,"$.realname") as realname,
get_json_object(response,"$.credentialno") as credentialno,
get_json_object(response,"$.expiretime") as expiretime,
get_json_object(response,"$.targetreporttype") as targetreporttype,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from `lovina_s3_etl`.`t_task_1`
where status=40 and channel=290  and (response is null or response ='' or substr(response,1,1)='{' ) and  id<11773266 
and  customer_id is not null 
""")
thirdparty_star_detector_apply_in_7_days.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_star_detector_apply_in_7_days/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
thirdparty_star_detector_apply_in_7_days_detail=spark.sql("""
select 
cast(a.customer_id as string) as customer_id,
cast(a.loan_id as string) as loan_id,
cast(a.create_time as string) as create_time,
a.thirdparty_id,query_key,
get_json_object(concat(a.reportDetailList,'}'),'$.reporttype') as reporttype,
get_json_object(concat(a.reportDetailList,'}'),'$.resulttype') as resulttype,
get_json_object(concat(a.reportDetailList,'}'),'$.resultdata') as resultdata,
year(to_date(create_time)) as year,
if(length(MONTH(to_date(create_time)))=1,concat("0",MONTH(to_date(create_time))),MONTH(to_date(create_time))) as month,
if(length(day(to_date(create_time)))=1,concat("0",day(to_date(create_time))),day(to_date(create_time))) as day
from (
select customer_id,loan_id,create_time,thirdparty_id,query_key,reportDetailList
from `lovina_etl_s3`.`t_task_1`a
LATERAL VIEW explode(split(substr(get_json_object(response,"$.reportDetailList"),2,length(get_json_object(response,"$.reportDetailList"))-2),'},')) as reportDetailList
where  status=40 and channel=290  and (response is null or response ='' or substr(response,1,1)='{') and  id<11773266 
and  customer_id is not null 
) a 
""")
thirdparty_star_detector_apply_in_7_days_detail.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_star_detector_apply_in_7_days_detail/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [1]:
spark.catalog.refreshTable("`lovina_etl_s3`.`t_task_1` ")
thirdparty_t_task=spark.sql("""
select 
id,
customer_id,
loan_id,
channel,
channel_id,
thirdparty_id,
status,
retry_times,
response,
cast(create_time as string) as create_time,
cast(update_time as string) as update_time,
client_id,
cast(query_key as string) as query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from `lovina_s3_etl`.`t_task_1`a
where id < 44982821  
and  status=40
and channel != 440
""")
thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1600223106183_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
thirdparty_tongdun_guard_risk_items_his=spark.sql("""
select 
id,
customer_id,
loan_id,
create_time,
query_key,
if(substr(get_json_object(response,"$.message"),-1)="}","",get_json_object(response,"$.message")) as message
concat(risk_items,'}') as risk_items
from `lovina_s3_etl`.`t_task`
LATERAL VIEW explode(split(substr(get_json_object(response,"$.result_desc.ANTIFRAUD.risk_items"),2,length(get_json_object(response,"$.result_desc.ANTIFRAUD.risk_items"))-2),'},')) as risk_items
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
 limit 100
""")
thirdparty_tongdun_guard_risk_items_his.write.mode("append").csv("s3://rupiahplus-data-warehouse/stream/thirdparty_his/thirdparty_tongdun_guard_risk_items_his/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
sc.method

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-6>

In [12]:
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
v1=spark.sql("""
select 
 *
from `lovina_s3_etl`.`t_task`
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
 limit 1
""")
noArr = spark.createDataFrame([[""]], T.ArrayType(T.StringType()))
noArr=noArr.select(noArr[0]).show()
v1=v1.collect()
for row in v1:
#     df = df.select(F.explode(df["result_desc.ANTIFRAUD.risk_items"]).alias("col"))
    print(row["id"])
    table=sc.parallelize(row)
    table=spark.read.json(table)
    table.printSchema()
#     df = table.select(F.explode(table["result_desc.ANTIFRAUD.risk_items"]).alias("col"))
#     col = df["col"]
#     type(col) 
#     df1 =df.withColumn("customer_id", lit(row["customer_id"]).cast(F.StringType())).withColumn("loan_id", lit(row["loan_id"]).cast(T.StringType())).withColumn("create_time", lit(row["create_time"]).cast(T.StringType())).withColumn("thirdparty_id", lit(row["thirdparty_id"])).withColumn("query_key", lit(row["query_key"]))
#     df1 =df1.withColumn("rule_id", col["rule_id"]).withColumn("score", col["score"].cast(T.IntegerType())).withColumn("decision", col["decision"]).withColumn("risk_name", col["risk_name"])
#     df1 =df1.withColumn("risk_detail",F.when(
#                         F.col('risk_detail').isNotNull(), F.col('risk_detail')).otherwise(lit(noArr))).drop("col")
#     df1.printSchema()
# #     df1.show();
#     df2=df1.select(F.explode(df1["risk_detail"]).alias("col2"))
#     col2 = df2["col2"]
#     df3 =df2.withColumn("rule_id", col2["type"])
#     df3.show()
#     for row2 in df1:
#         df2=df1.select(F.explode(row2["risk_detail"]).alias("col2"))
#         df2.show()
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|value|
+-----+
|   []|
+-----+

926310
root
 |-- _corrupt_record: string (nullable = true)
 |-- id: string (nullable = true)
 |-- result_desc: struct (nullable = true)
 |    |-- ANTIFRAUD: struct (nullable = true)
 |    |    |-- final_decision: string (nullable = true)
 |    |    |-- final_score: long (nullable = true)
 |    |    |-- risk_items: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- decision: string (nullable = true)
 |    |    |    |    |-- risk_detail: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- risk_details: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |    |    |    |    |-- field_type: string (nullable = true)
 |    |    |    |    |    |    |    |    |-- hit_rule_t

In [4]:
v1=spark.sql("""
select 
testudf()
from `lovina_s3_etl`.`t_task`
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
 limit 10
""").show()

VBox()

An error was encountered:
Session 7 did not reach idle status in time. Current status is busy.


In [2]:
v1=spark.sql("""
select 
*
from `lovina_s3_etl`.`t_task`
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and customer_id is not null
 limit 10
""")
v1.write.mode("append").orc("s3://rupiahplus-data-warehouse/stream/thirdparty_his/json/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
from urllib.request import urlopen
import json
from pprint import pprint

# a='{"success":true,"id":"WF2019021715404113217023","result_desc":{"ANTIFRAUD":{"final_score":0,"risk_items":[{"rule_id":"23374771","score":0,"decision":"Accept","risk_name":"Borrower\\'s,name_same_with_KTP_name","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"},{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"}],"type":"mh_detail"}]},{"rule_id":"23374772","score":0,"decision":"Accept1111","risk_name":"Borrower_name_same_with_KTP_name111111","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name111111","hit_rule_type":"Identity verification11111","field_type":"name11111","input_value":"Mertina Zai1111"},{"description":"Borrowers name is same with KTP name1111","hit_rule_type":"Identity verification11111","field_type":"name1111","input_value":"Mertina Zai11111"}],"type":"mh_detail"}]}],"final_decision":"Accept"}}}'
# def tongdun_guard_json(b,typeValue):
#     return getJsonArray(b,typeValue);
def tongdun_guard_json_test(b,typeValue):
    risk_itemls=[];
    risk_detaillist=[];
    risk_detailslist=[];
    text = json.loads(b)
    for field in text.keys():
        if field=="risk_items":
            index=0;
            for v1 in text[field]:
                index=index+1;
                for field1 in list(v1):
                    if field1=='risk_detail':
                        for v2 in v1[field1]:
                            for risk_detailfield in  list(v2):
                                if risk_detailfield=='risk_details':
                                     for v3 in v2[risk_detailfield]:
                                         v3["rule_id"]=v1["rule_id"];
                                         v3["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                         risk_detailslist.append(json.dumps(v3));
                                     v2.pop(risk_detailfield);
                                v2["rule_id"]=v1["rule_id"];
                                v2["rule_detail_id"]=v1["rule_id"]+"_"+str(index);
                                risk_detaillist.append(json.dumps(v2))
                        v1.pop(field1);
                risk_itemls.append(json.dumps(v1))
            break;
        elif field!="risk_items" and type(text[field]) == dict:
            result_desc=json.dumps(text[field]);
            return tongdun_guard_json_test(result_desc,typeValue);
        else :
            continue
    if typeValue=='risk_items':
        return  risk_itemls
    elif typeValue=='risk_detail':
        return  risk_detaillist
    elif typeValue=='risk_details':
        return risk_detailslist;
# tongdun_guard_json(a,"risk_items");
spark.udf.register("tongdun_guard_json_test",tongdun_guard_json_test,T.ArrayType(T.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function tongdun_guard_json_test at 0x7f7394a38d90>

In [81]:
thirdparty_tongdun_guard_risk_details=spark.sql("""
select 
a.id,
a.customer_id,
a.loan_id,
a.create_time,
a.query_key,
get_json_object(risk_details,"$.rule_id") as rule_id,
get_json_object(risk_details,"$.rule_detail_id") as rule_detail_id,
regexp_replace(get_json_object(risk_details,"$.description"),"##","\\'") as description,
get_json_object(risk_details,"$.hit_rule_type") as hit_rule_type,
get_json_object(risk_details,"$.field_type") as field_type,
get_json_object(risk_details,"$.input_value") as input_value
from (
select 
id,
customer_id,
loan_id,
create_time,
query_key, 
risk_details as risk_details
from `lovina_s3_etl`.`t_task`
LATERAL VIEW explode(tongdun_guard_json_test(response,"risk_details")) as risk_details
where status=40 
and channel=40  
and (response is null or response ='' or substr(response,1,1)='{' )
and (client_id=0 or client_id is null )
and id=1831209
 limit 100)  a
""").show(10,False)
# thirdparty_tongdun_guard_risk_details.write.mode("overwrite").csv("s3://rupiahplus-data-warehouse/stream/thirdparty_his/json/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+-------+--------------------------+---------+--------+--------------+--------------------------------------+-----------------------+----------------------+-------------------+
|id     |customer_id|loan_id|create_time               |query_key|rule_id |rule_detail_id|description                           |hit_rule_type          |field_type            |input_value        |
+-------+-----------+-------+--------------------------+---------+--------+--------------+--------------------------------------+-----------------------+----------------------+-------------------+
|1831209|2986252    |5785628|2019-03-15 06:05:02.233743|null     |23375031|23375031_1    |Borrower applied many times in 14 days|Apply behavior analysis|applicatioin frequency|null               |
|1831209|2986252    |5785628|2019-03-15 06:05:02.233743|null     |23374771|23374771_2    |Borrower's name is same with KTP name |Identity verification  |name                  |martha eka prasetya|
+-------+------

In [70]:
from urllib.request import urlopen
import json
from pprint import pprint
from pyspark.sql import types as T

# a='{"success":true,"id":"WF2019021715404113217023","result_desc":{"ANTIFRAUD":{"final_score":0,"risk_items":[{"rule_id":"23374771","score":0,"decision":"Accept","risk_name":"Borrower\\'s,name_same_with_KTP_name","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"},{"description":"Borrowers name is same with KTP name","hit_rule_type":"Identity verification","field_type":"name","input_value":"Mertina Zai"}],"type":"mh_detail"}]},{"rule_id":"23374772","score":0,"decision":"Accept1111","risk_name":"Borrower_name_same_with_KTP_name111111","risk_detail":[{"risk_details":[{"description":"Borrowers name is same with KTP name111111","hit_rule_type":"Identity verification11111","field_type":"name11111","input_value":"Mertina Zai1111"},{"description":"Borrowers name is same with KTP name1111","hit_rule_type":"Identity verification11111","field_type":"name1111","input_value":"Mertina Zai11111"}],"type":"mh_detail"}]}],"final_decision":"Accept"}}}'
# def tongdun_guard_json(b,typeValue):
#     return getJsonArray(b,typeValue);
def tongdun_guard_json_test1(b,typeValue):
    a=["a","v","c"]
    return a
# tongdun_guard_json(a,"risk_items");
spark.udf.register("tongdun_guard_json_test1",tongdun_guard_json_test1,T.ArrayType(T.StructType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function tongdun_guard_json_test1 at 0x7f739726c510>

In [72]:
thirdparty_tongdun_guard_risk_details=spark.sql("""
select explode(a) from (
select tongdun_guard_json_test1("a","b") as a
) b
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
|col|
+---+
|  a|
|  v|
|  c|
+---+

In [8]:
spark.sql("""
select 
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from `lovina_s3_etl`.`t_task_1`a
limit 100
""").show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+
|year|month|day|
+----+-----+---+
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
|2020|01   |07 |
+----+-----+---+
only showing top 10 rows

In [ ]:
select 
 count(1)
from 
lovina_s3_etl.t_task_1 a 
left join 
  thirdparty_etl.thirdparty_t_task  b 
  on a.id =b.id 
  and cast(substr(b.create_time, 1, 19) AS timestamp) < cast('2020-01-15 00:00:00'  AS timestamp) 
  where a.channel!=440 
  and a.status=40
  and a.create_time < cast('2020-01-15 00:00:00'  AS timestamp) 
  and b.id is null

In [4]:
spark.catalog.refreshTable("`lovina_s3_etl`.`t_task_1` ")
spark.catalog.refreshTable("`lovina_s3_etl`.`t_task_1` ")
thirdparty_t_task=spark.sql("""
select 
a.id,
a.customer_id,
a.loan_id,
a.channel,
a.channel_id,
a.thirdparty_id,
a.status,
a.retry_times,
a.response,
cast(a.create_time as string) as create_time,
cast(a.update_time as string) as update_time,
a.client_id,
cast(a.query_key as string) as query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.create_time)))=1,concat("0",MONTH(to_date(a.create_time))),MONTH(to_date(a.create_time))) as month,
if(length(day(to_date(a.create_time)))=1,concat("0",day(to_date(a.create_time))),day(to_date(a.create_time))) as day
from 
lovina_s3_etl.t_task_1 a 
left join 
  thirdparty_etl.thirdparty_t_task  b 
  on a.id =b.id 
  and cast(substr(b.create_time, 1, 19) AS timestamp) < cast('2020-01-15 00:00:00'  AS timestamp) 
  where a.channel!=440 
  and a.status=40
  and a.create_time < cast('2020-01-15 00:00:00'  AS timestamp) 
  and b.id is null
""")
thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
thirdparty_t_task=spark.sql("""
select 
a.id,
a.customer_id,
a.loan_id,
a.channel,
a.channel_id,
a.thirdparty_id,
a.status,
a.retry_times,
a.response,
cast(a.create_time as string) as create_time,
cast(a.update_time as string) as update_time,
a.client_id,
cast(a.query_key as string) as query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.update_time)))=1,concat("0",MONTH(to_date(a.update_time))),MONTH(to_date(a.update_time))) as month,
if(length(day(to_date(a.update_time)))=1,concat("0",day(to_date(a.update_time))),day(to_date(a.update_time))) as day
from 
lovina_s3_etl.t_task_1 a 
left join 
  thirdparty_etl.thirdparty_t_task  b 
  on a.id =b.id 
  where a.channel!=440 
  and a.status=40
  and b.id is null
""")
#  and cast(substr(a.create_time, 1, 19) AS timestamp) >= cast('2020-03-02 00:00:00'  AS timestamp) 
#  and cast(substr(a.create_time, 1, 19) AS timestamp) < cast('2020-03-02 10:00:00'  AS timestamp)
#  and a.create_time >= cast('2020-02-05 00:00:00'  AS timestamp) 
#   and a.create_time < cast('2020-02-06 00:00:00'  AS timestamp) 
thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
thirdparty_t_task=spark.sql("""
select 
*
from 
lovina_s3_etl.t_task_1 a 
left join 
  thirdparty_etl.thirdparty_t_task  b 
  on a.id =b.id 
  where a.channel!=440 
  and a.status=40
  and b.id is null
  and a.id<17082526
  order by a.create_time desc
   limit 1000
""").show(1)
#  and cast(substr(b.create_time, 1, 19) AS timestamp) >= cast('2020-02-05 00:00:00'  AS timestamp) 
#   and cast(substr(b.create_time, 1, 19) AS timestamp) < cast('2020-02-06 00:00:00'  AS timestamp)
#  and a.create_time >= cast('2020-02-05 00:00:00'  AS timestamp) 
#   and a.create_time < cast('2020-02-06 00:00:00'  AS timestamp) 
# thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")

In [ ]:
a.id,
a.customer_id,
a.loan_id,
a.channel,
a.channel_id,
a.thirdparty_id,
a.status,
a.retry_times,
a.response,
cast(a.create_time as string) as create_time,
cast(a.update_time as string) as update_time,
a.client_id,
cast(a.query_key as string) as query_key,
year(to_date(a.create_time)) as year,
if(length(MONTH(to_date(a.update_time)))=1,concat("0",MONTH(to_date(a.update_time))),MONTH(to_date(a.update_time))) as month,
if(length(day(to_date(a.update_time)))=1,concat("0",day(to_date(a.update_time))),day(to_date(a.update_time))) as day

In [1]:
from datetime import datetime, timedelta
print(datetime(2021,4,22))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1619058808510_0013,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2021-04-22 00:00:00

In [3]:
spark.sql("""
     select count(1) from thirdparty_etl.thirdparty_t_task
     where year='2021' and month='07' and day >='15'
     
""").show(1000,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|222649  |
+--------+

In [5]:
thirdparty_t_task=spark.sql("""
 with exist_id as (
      select id from thirdparty_etl.thirdparty_t_task
     where year='2021' and month='07' and day >='15'
     )
 
    select  a.id,
    a.customer_id,
    a.loan_id,
    a.channel,
    a.channel_id,
    a.thirdparty_id,
    a.status,
    a.retry_times,
    a.response,
    cast(a.create_time as string) as create_time,
    cast(a.update_time as string) as update_time,
    a.client_id,
    cast(a.query_key as string) as query_key,
    a.tag,
    '2021' as year,
    '07' as month,
    '01' as day 
    from lovina_s3_etl.t_task_1 a
    left join exist_id b
     on a.id =b.id
    where b.id is  null
    and  a.status = 40
    and a.channel != 440
 


""")
thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")
#   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
thirdparty_t_task=spark.sql("""
 
    select  a.id,
    a.customer_id,
    a.loan_id,
    a.channel,
    a.channel_id,
    a.thirdparty_id,
    40 as status,
    a.retry_times,
    a.response,
    cast(a.create_time as string) as create_time,
    cast(a.update_time as string) as update_time,
    a.client_id,
    cast(a.query_key as string) as query_key,
    '2021' as year,
    '07' as month,
    '02' as day 
    from lovina_etl_s3.t_task_1 a
    where 1=1
    and a.channel = 10015  and a.client_id in (0,10,1000)
    and  a.status <> 40
    and a.channel != 440
 


""")
thirdparty_t_task.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/thirdparty/thirdparty_t_task/")
#   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…